# Stock Price Prediction Model

## Dependencies

### Library Installation (if needed)

In [1]:
#! pip install -Ur requirements.txt

### Importing Required Libraries

In [2]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
import gradio as gr
import ast
import warnings
from methods.model_methods import *
from datetime import datetime
from tqdm import tqdm
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Variables

In [3]:
symbol_list = input("Symbols: ('simple', 'filtered' or 'all')") # 'simple' or 'all'. simple are the tickers from the screener notebook and all are all of the tickers in 'filtered_tickers'. You may also an 'Int' to get a % amount of random tickers from 'all'
if symbol_list:
    build_new_dataset = True
else:
    build_new_dataset = False
    symbol_list = 'filtered'
minimum_feature_threshold = 0.6
outlier = 3
tree_amount = input("Tree amount:")
if tree_amount:
    tree_amount = int(tree_amount)
    train_new_model = True
else:
    train_new_model = False
if build_new_dataset or train_new_model:
    debugging = {'True': True, 'False': False}.get(input('Debug? (Bool)'))
else:
    debugging = False

In [4]:
symbols = pd.read_csv('../data/tickers/simple_tickers.csv')['Ticker'].tolist()
if symbol_list == 'filtered':
    symbols = symbols + pd.read_csv('../data/tickers/model_filtered_tickers.csv')['Ticker'].tolist()
elif symbol_list == 'all':
    symbols = symbols + pd.read_csv('../data/tickers/tickers.csv')['Ticker'].tolist()
elif symbol_list.isdigit():
    all_symbols = pd.read_csv('../data/tickers/tickers.csv')['Ticker'].tolist()
    num_symbols = max(1, round(len(all_symbols) * (int(symbol_list) / 100)))  
    symbols = symbols + pd.read_csv("../data/tickers/model_filtered_tickers.csv")["Ticker"].tolist()
    symbols = symbols + np.random.choice(all_symbols, num_symbols, replace=False).tolist()
symbols = pd.Series(symbols).unique()

## Data Preparation

#### Test load

In [5]:
test_stock = Stock("MSFT").get_df_financials()
if debugging:
    display(test_stock)

,Ticker,Name,Date,Earn Index,Sector,Industry,3M Future Change,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,...,RawMaterials,Receivables,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,GrossAccountsReceivable,CashCashEquivalentsAndShortTermInvestments,OtherShortTermInvestments,CashAndCashEquivalents,CashEquivalents,CashFinancial
0,MSFT,Microsoft Corporation,2025-03-31,0,Technology,Software - Infrastructure,NaN,69660000.0,0.180,4.032400e+10,...,3.270000e+08,5.170000e+10,5.170000e+10,-695000000.0,5.239500e+10,7.961200e+10,5.078400e+10,2.882800e+10,1.814800e+10,1.068000e+10
1,MSFT,Microsoft Corporation,2024-12-31,1,Technology,Software - Infrastructure,-0.091444,-203220000.0,0.180,3.791500e+10,...,3.450000e+08,4.818800e+10,4.818800e+10,-662000000.0,4.885000e+10,7.155100e+10,5.406900e+10,1.748200e+10,7.835000e+09,9.647000e+09
2,MSFT,Microsoft Corporation,2024-09-30,2,Technology,Software - Infrastructure,-0.010736,57190000.0,0.190,3.793300e+10,...,4.880000e+08,4.414800e+10,4.414800e+10,-647000000.0,4.479500e+10,7.842900e+10,5.758900e+10,2.084000e+10,1.025200e+10,1.058800e+10
3,MSFT,Microsoft Corporation,2024-06-30,3,Technology,Software - Infrastructure,-0.040628,-99918000.0,0.182,1.335580e+11,...,3.940000e+08,5.692400e+10,5.692400e+10,-830000000.0,5.775400e+10,7.553100e+10,5.721600e+10,1.831500e+10,6.744000e+09,1.157100e+10
5,MSFT,Microsoft Corporation,2023-06-30,5,Technology,Software - Infrastructure,-0.070833,-2850000.0,0.190,1.051550e+11,...,7.090000e+08,4.868800e+10,4.868800e+10,-650000000.0,4.933800e+10,1.112560e+11,7.655200e+10,3.470400e+10,2.622600e+10,8.478000e+09
6,MSFT,Microsoft Corporation,2022-06-30,6,Technology,Software - Infrastructure,-0.073301,43754000.0,0.131,9.990500e+10,...,1.144000e+09,4.426100e+10,4.426100e+10,-633000000.0,4.489400e+10,1.047490e+11,9.081800e+10,1.393100e+10,5.673000e+09,8.258000e+09


#### Download annual financial data

In [6]:
if build_new_dataset:
    df = pd.DataFrame()
    filtered_pd = pd.read_csv("../data/tickers/model_filtered_tickers.csv")
    for symbol in tqdm(symbols, smoothing=0):
        ticker_df = pd.DataFrame()
        ticker = yf.Ticker(symbol)
        stock = Stock(symbol)
        try:
            ticker_df = Stock(symbol).get_df_financials()
            if ticker_df.isna().sum().sum() < round(test_stock.shape[1] * minimum_feature_threshold):
                if symbol not in filtered_pd["Ticker"].tolist():
                    filtered_pd = pd.concat([filtered_pd, pd.DataFrame([{"Ticker": symbol}])])
                imputer = SimpleImputer()
                for column in ticker_df.columns.drop(["Ticker", "Name", "Date", "3M Future Change", "Sector", "Industry"]):
                    if not ticker_df[column].isna().all():
                        ticker_df[column] = imputer.fit_transform(ticker_df[[column]])
            else:
                if symbol in filtered_pd["Ticker"].tolist():
                    filtered_pd = filtered_pd[filtered_pd["Ticker"] != symbol]
                    if debugging:
                        print(f"Removed {symbol} from filtered tickers. Datapoints: {ticker_df.isna().sum().sum()}, Needed: {round(test_stock.shape[1] * minimum_feature_threshold)}")
                continue
            df = pd.concat([df, ticker_df], ignore_index=True)
        except Exception as error:
            if symbol in filtered_pd["Ticker"].tolist():
                filtered_pd = filtered_pd[filtered_pd["Ticker"] != symbol]
                if debugging:
                    print(f"Removed {symbol} from filtered tickers because an exception was raised \n {error}")
            else:
                if debugging:
                    print(f"{symbol}: exception raised: {error}")
            continue
    filtered_pd.to_csv("../data/tickers/model_filtered_tickers.csv", index=False)
    df.to_csv("../data/model_results/earnings_data.csv", index=False)
else:
    df = pd.read_csv("../data/model_results/earnings_data.csv")

  3%|▎         | 27/1010 [02:02<1:14:34,  4.55s/it]

Removed BAVA.CO  from filtered tickers because an exception was raised 
 "['Ticker', 'Name', 'Date', '3M Future Change', 'Sector', 'Industry'] not found in axis"


  3%|▎         | 32/1010 [02:22<1:12:25,  4.44s/it]

Removed NETC.CO from filtered tickers. Datapoints: 92, Needed: 79


  4%|▍         | 39/1010 [02:56<1:13:22,  4.53s/it]

...

 97%|█████████▋| 984/1010 [1:12:57<01:55,  4.45s/it]

Removed GLEN.L from filtered tickers. Datapoints: 138, Needed: 79


100%|█████████▉| 1009/1010 [1:14:51<00:04,  4.45s/it]

Removed CSU.TO from filtered tickers. Datapoints: 88, Needed: 79


100%|██████████| 1010/1010 [1:14:56<00:00,  4.45s/it]


### Short visualisation

In [7]:
if debugging:
    display(df)

,Ticker,Name,Date,Earn Index,Sector,Industry,3M Future Change,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,BAESY,BAE Systems PLC,2024-12-31,1.0,Industrials,Aerospace & Defense,0.451540,-2.520000e+06,0.180000,3.986000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAESY,BAE Systems PLC,2023-12-31,3.0,Industrials,Aerospace & Defense,0.216240,-5.567000e+07,0.190000,3.740000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAESY,BAE Systems PLC,2022-12-31,4.0,Industrials,Aerospace & Defense,0.171293,7.744344e+07,0.158371,2.570000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAESY,BAE Systems PLC,2021-12-31,5.0,Industrials,Aerospace & Defense,0.257805,2.160000e+07,0.180000,3.005000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LLY,Eli Lilly and Company,2025-03-31,0.0,Healthcare,Drug Manufacturers - General,NaN,-3.541868e+08,0.202000,5.916000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4572,FM.TO,FIRST QUANTUM MINERALS LTD,2024-09-30,2.0,Basic Materials,Copper,0.010846,-4.050000e+06,0.150000,5.230000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4573,FM.TO,FIRST QUANTUM MINERALS LTD,2024-06-30,3.0,Basic Materials,Copper,0.054535,-1.095000e+07,0.150000,3.560000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4574,FM.TO,FIRST QUANTUM MINERALS LTD,2023-12-31,5.0,Basic Materials,Copper,0.317647,-4.064000e+08,0.400000,2.292000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4575,FM.TO,FIRST QUANTUM MINERALS LTD,2022-12-31,6.0,Basic Materials,Copper,0.098268,4.048000e+07,0.220000,3.099000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Impution and encoding

In [8]:
imputer = SimpleImputer()
scaler = StandardScaler()
for column in df.columns.drop(["Ticker", "Name", "Date", "Earn Index", "3M Future Change", "Sector", "Industry"]):
    df[column] = imputer.fit_transform(df[[column]])
    scaler.fit(df[[column]])
    df[column] = scaler.transform(df[[column]])

le = LabelEncoder()
for column in ["Ticker", "Sector", "Industry"]:
    df[column] = df[column].astype(str)
    le.fit(df[column])
    df[column] = le.transform(df[column])

### Splitting

In [10]:
pred_data = df.loc[df["Earn Index"] == 0].copy()
test_data  = df.loc[df["Earn Index"] == 1].copy()
train_data = df.loc[~df["Earn Index"].isin([0, 1])].copy()

pred_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
train_data.reset_index(drop=True, inplace=True)

if debugging:
    print('Prediction Data:')
    display(pred_data)
    print("Test Data:")
    display(test_data)
    print('Training Data:')
    display(train_data)

Prediction Data:


,Ticker,Name,Date,Earn Index,Sector,Industry,3M Future Change,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,464,Eli Lilly and Company,2025-03-31,0.0,6,30,NaN,-0.407855,0.186254,0.237679,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,185,Danske Bank A/S,2025-03-31,0.0,5,11,NaN,-0.003093,0.621869,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,435,Jyske Bank A/S,2025-06-30,0.0,5,11,NaN,-0.003093,0.822176,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,560,"Netflix, Inc.",2025-06-30,0.0,1,37,NaN,-0.003093,-0.488949,0.369670,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,738,Scandinavian Tobacco Group A/S,2025-03-31,0.0,3,107,NaN,-0.021440,0.491184,-0.169067,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,123,CWLTH BANK FPO [CBA],2025-06-30,0.0,5,10,NaN,-0.192778,1.297072,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
531,654,ProSiebenSat.1 Media SE N,2023-12-31,0.0,1,16,NaN,-0.011504,-0.009773,-0.179712,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
532,441,Kinnevik AB ser. B,2025-06-30,0.0,5,6,NaN,-0.003093,-2.013602,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
533,811,WASTE CONNECTIONS INC,2025-06-30,0.0,7,116,NaN,-0.004263,0.752553,-0.136507,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


Test Data:


,Ticker,Name,Date,Earn Index,Sector,Industry,3M Future Change,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,61,BAE Systems PLC,2024-12-31,1.0,7,1,0.451540,-0.005973,-0.053334,0.096969,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,464,Eli Lilly and Company,2024-12-31,1.0,6,30,0.044842,-0.793256,-0.216690,1.222071,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,185,Danske Bank A/S,2024-12-31,1.0,5,11,0.176085,-0.011424,0.632759,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,118,Carlsberg B A/S,2024-12-31,1.0,3,12,0.309414,-0.116230,0.142692,0.994522,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,435,Jyske Bank A/S,2025-03-31,1.0,5,11,0.163339,-0.003093,0.807005,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,441,Kinnevik AB ser. B,2025-03-31,1.0,5,6,0.187961,-0.003093,-2.013602,0.000000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
669,472,LONDON STOCK EXCHANGE GROUP PLC,2024-12-31,1.0,5,40,0.027027,-0.048421,0.905019,0.114539,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
670,304,FRESNILLO PLC ORD USD0.50,2024-12-31,1.0,0,74,0.533387,-0.055384,1.253511,-0.077882,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
671,811,WASTE CONNECTIONS INC,2025-03-31,1.0,7,116,-0.091643,-0.004771,0.469404,-0.143314,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


Training Data:


,Ticker,Name,Date,Earn Index,Sector,Industry,3M Future Change,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,61,BAE Systems PLC,2023-12-31,3.0,7,1,0.216240,-0.066712,0.055569,0.079034,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,61,BAE Systems PLC,2022-12-31,4.0,7,1,0.171293,0.085409,-0.288882,-0.006267,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,61,BAE Systems PLC,2021-12-31,5.0,7,1,0.257805,0.021591,-0.053334,0.025448,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,464,Eli Lilly and Company,2024-09-30,2.0,6,30,-0.125086,-1.245866,2.222754,0.174068,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,464,Eli Lilly and Company,2024-06-30,3.0,6,30,-0.029119,-0.136669,-0.310010,0.160864,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3364,292,FIRST QUANTUM MINERALS LTD,2024-09-30,2.0,0,26,0.010846,-0.007722,-0.380046,-0.155507,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3365,292,FIRST QUANTUM MINERALS LTD,2024-06-30,3.0,0,26,0.054535,-0.015607,-0.380046,-0.167682,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3366,292,FIRST QUANTUM MINERALS LTD,2023-12-31,5.0,0,26,0.317647,-0.467524,2.342548,-0.026535,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3367,292,FIRST QUANTUM MINERALS LTD,2022-12-31,6.0,0,26,0.098268,0.043167,0.382281,0.032301,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


### Labeling

In [11]:
training_columns = train_data.columns.drop(["Name", "Sector", "Industry", "Date", '3M Future Change'])
label_columns = ['3M Future Change']
X_pred = pred_data[training_columns]
X_test = test_data[training_columns]
y_test = test_data[label_columns]
X_train = train_data[training_columns]
y_train = train_data[label_columns]
if debugging:
    print("X_pred:")
    display(X_pred)
    print("X_test:")
    display(X_test)
    print("y_test:")
    display(y_test)
    print("X_train:")
    display(X_train)
    print("y_train:")
    display(y_train)

X_pred:


,Ticker,Earn Index,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,NetIncomeFromContinuingOperationNetMinorityInterest,ReconciledDepreciation,ReconciledCostOfRevenue,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,464,0.0,-0.407855,0.186254,0.237679,-6.916848e-01,-6.893223e-01,0.059503,-0.067045,-2.132019e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,185,0.0,-0.003093,0.621869,0.000000,-1.060253e+00,-1.057767e+00,0.193615,0.919742,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,435,0.0,-0.003093,0.822176,0.000000,-8.434582e-03,-6.301482e-03,-0.006634,-0.153817,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,560,0.0,-0.003093,-0.488949,0.369670,-1.485529e-18,7.425151e-18,0.075882,0.980186,-3.396913e-03,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,738,0.0,-0.021440,0.491184,-0.169067,-2.031789e-02,-1.818080e-02,-0.061640,-0.173764,-2.887858e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,123,0.0,-0.192778,1.297072,0.000000,-2.102115e-01,-2.080107e-01,0.389391,0.216889,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
531,654,0.0,-0.011504,-0.009773,-0.179712,-7.637044e-03,-6.301482e-03,-0.069492,-0.138636,-1.734344e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
532,441,0.0,-0.003093,-2.013602,0.000000,-1.485529e-18,7.425151e-18,-0.035938,0.000000,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
533,811,0.0,-0.004263,0.752553,-0.136507,5.909126e-03,8.037410e-03,-0.050958,-0.114148,-2.694462e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


X_test:


,Ticker,Earn Index,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,NetIncomeFromContinuingOperationNetMinorityInterest,ReconciledDepreciation,ReconciledCostOfRevenue,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,61,1.0,-0.005973,-0.053334,0.096969,1.933403e-03,4.063022e-03,0.023564,0.121860,2.509640e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,464,1.0,-0.793256,-0.216690,1.222071,-1.663524e+00,-1.660835e+00,0.409836,0.328799,2.058585e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,185,1.0,-0.011424,0.632759,0.000000,-4.446895e-03,-2.315134e-03,0.993181,0.103947,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,118,1.0,-0.116230,0.142692,0.994522,-1.918682e-01,-1.896735e-01,0.242872,1.119212,2.385201e+00,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,435,1.0,-0.003093,0.807005,0.000000,-8.434582e-03,-6.301482e-03,-0.007753,-0.149566,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,441,1.0,-0.003093,-2.013602,0.000000,-1.485529e-18,7.425151e-18,-0.199815,0.000000,-6.452061e-17,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
669,472,1.0,-0.048421,0.905019,0.114539,-5.150159e-02,-4.935404e-02,-0.033298,0.535981,-2.843206e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
670,304,1.0,-0.055384,1.253511,-0.077882,-5.330602e-02,-5.115786e-02,-0.057640,-0.019055,-2.114488e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
671,811,1.0,-0.004771,0.469404,-0.143314,4.948094e-03,7.076701e-03,-0.053140,-0.119524,-2.763073e-01,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


y_test:


,3M Future Change
0,0.451540
1,0.044842
2,0.176085
3,0.309414
4,0.163339
...,...
668,0.187961
669,0.027027
670,0.533387
671,-0.091643


X_train:


,Ticker,Earn Index,TaxEffectOfUnusualItems,TaxRateForCalcs,NormalizedEBITDA,TotalUnusualItems,TotalUnusualItemsExcludingGoodwill,NetIncomeFromContinuingOperationNetMinorityInterest,ReconciledDepreciation,ReconciledCostOfRevenue,...,LiabilitiesHeldforSaleNonCurrent,DuetoRelatedPartiesNonCurrent,OccupancyAndEquipment,ProfessionalExpenseAndContractServicesExpense,PolicyholderBenefitsGross,PolicyholderBenefitsCeded,DepletionIncomeStatement,OtherNonInterestExpense,SecuritiesAmortization,ExciseTaxes
0,61,3.0,-0.066712,0.055569,0.079034,-0.109323,-0.107156,0.019135,0.029866,0.168966,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
1,61,4.0,0.085409,-0.288882,-0.006267,0.202514,0.204576,0.007235,0.024401,0.116263,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
2,61,5.0,0.021591,-0.053334,0.025448,0.055368,0.057480,0.014706,0.005274,0.107265,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3,464,2.0,-1.245866,2.222754,0.174068,-1.107281,-1.104779,-0.020535,-0.065831,-0.216815,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
4,464,3.0,-0.136669,-0.310010,0.160864,-0.290444,-0.288216,0.068795,-0.081740,-0.216855,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3364,292,2.0,-0.007722,-0.380046,-0.155507,-0.003251,-0.001119,-0.059113,-0.159282,-0.308000,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3365,292,3.0,-0.015607,-0.380046,-0.167682,-0.021594,-0.019456,-0.066002,-0.162621,-0.302926,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3366,292,5.0,-0.467524,2.342548,-0.026535,-0.397633,-0.395369,-0.106625,0.132789,-0.014310,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0
3367,292,6.0,0.043167,0.382281,0.032301,0.080890,0.082993,-0.017685,0.165883,0.003415,...,-4.759382e-16,-1.324493e-15,6.922852e-17,0.0,1.138282e-15,0.0,0.0,-8.378651e-17,3.917480e-15,0.0


y_train:


,3M Future Change
0,0.216240
1,0.171293
2,0.257805
3,-0.125086
4,-0.029119
...,...
3364,0.010846
3365,0.054535
3366,0.317647
3367,0.098268


## Model Training

In [12]:
if train_new_model:
    model = RandomForestRegressor(
        n_estimators=tree_amount,
        verbose=True
    )
    model.fit(X_train, y_train.values.ravel())

## Testing and benchmarking

In [13]:
if train_new_model:
    y_test_pred = model.predict(X_test)

    for i, target in enumerate(['3M Future Change']):
        y_test_actual = y_test[target].values
        if getattr(y_test_pred, "ndim", 1) == 1:
            y_test_pred_target = y_test_pred
        else:
            y_test_pred_target = y_test_pred[:, i]

        plt.figure(figsize=(11, 6))
        plt.scatter(y_test_actual, y_test_pred_target, alpha=0.7, color='blue', label='Predictions')
        plt.plot([y_test_actual.min(), y_test_actual.max()], [y_test_actual.min(), y_test_actual.max()], 
            color='red', linestyle='--', label='Perfect Fit')
        plt.title(f'Predicted vs Actual Values ({target})')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.legend()
        plt.grid(True)
        plt.show()

        mae = mean_absolute_error(y_test_actual, y_test_pred_target)
        mse = mean_squared_error(y_test_actual, y_test_pred_target)
        r2 = r2_score(y_test_actual, y_test_pred_target)

        print(f'{target} - R²: {r2:.4f}')
        print(f'{target} - MSE: {mse:.4f}')
        print(f'{target} - MAE: {mae:.4f}')

    # Overall metrics (single target)
    overall_pred = y_test_pred if getattr(y_test_pred, "ndim", 1) == 1 else y_test_pred[:, 0]
    mae = mean_absolute_error(y_test['3M Future Change'].values, overall_pred)
    mse = mean_squared_error(y_test['3M Future Change'].values, overall_pred)
    r2 = r2_score(y_test['3M Future Change'].values, overall_pred)

    print('\nOverall Scores:')
    print(f'Mean - R²: {r2:.4f}')
    print(f'Mean - MSE: {mse:.4f}')
    print(f'Mean - MAE: {mae:.4f}')

### Log test results

In [14]:
if train_new_model:
    test_results = pd.DataFrame({
        'R²': r2,
        'MSE': mse,
        'MAE': mae,
        'symbol_list': symbol_list,
        "iterations": iterations,
        'hidden_layer_sizes': [model.model.hidden_layer_sizes],
        'max_iter': model.model.max_iter,
        'n_iter_no_change': model.model.n_iter_no_change,
        'learning_rate': model.model.learning_rate,
        'learning_rate_init': model.model.learning_rate_init,
        'batch_size': model.model.batch_size,
        'tol': model.model.tol,
        'alpha': model.model.alpha,
        'shuffle': model.model.shuffle,
    })
    test_results.to_csv('../data/model_results/test_results.csv', mode='a', index=False)

    # save model as new best if results are better than the current one
    best_r2 = pd.read_csv('../data/model_results/best_model_results.csv').loc[0, 'R²']
    if r2 > best_r2:
        print(f'Old best R²: {best_r2}')
        print(f'New best R²: {r2}')
        print('Saving new best model...')
        test_results.to_csv('../data/model_results/best_model_results.csv', mode='w', index=False)
        with open('../models/best_model.pkl','wb') as f:
            pickle.dump(model,f)
        

## Predictions on latest data

In [15]:
best_r2 = pd.read_csv('../data/model_results/best_model_results.csv').loc[0, 'R²']

with open('../models/best_model.pkl', 'rb') as f:
    model = pickle.load(f)
    print(f'Best model R²: {best_r2}')


FileNotFoundError: [Errno 2] No such file or directory: '../models/best_model.pkl'

In [ ]:
df_raw = pd.read_csv('../data/model_results/earnings_data.csv')

results = []
for i in range(len(X_pred)):
    y_pred = model.predict(X_pred.iloc[[i]])[0]
    y_pred_3m, y_pred_6m, y_pred_9m, y_pred_1y = y_pred
    avg = (y_pred_3m + y_pred_6m + y_pred_9m + y_pred_1y) / 4
    results.append({
        'Ticker': df_raw.loc[i*4, 'Ticker'],
        'Name': df_raw.loc[i*4, 'Name'],
        'mean (%)': avg * 100,
        '3m (%)': y_pred_3m * 100,
        '6m (%)': y_pred_6m * 100,
        '9m (%)': y_pred_9m * 100,
        '1y (%)': y_pred_1y * 100
    })

results_df = pd.DataFrame(results)
results_df

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ExciseTaxes


In [ ]:
def predict_ticker(ticker_str):
    try:
        ticker_str = str(ticker_str).upper()
        ticker_str = ticker_str.replace("'", "")
        ticker_str = ticker_str.replace('"', "")
        row = results_df[results_df['Ticker'] == ticker_str]
        if row.empty:
            return (f"Not enough data for this stock at this moment \n Try another", "", "", "", "")
        row = row.iloc[0]
        return (
            f"{row['3m (%)']:.2f}",
            f"{row['6m (%)']:.2f}",
            f"{row['9m (%)']:.2f}",
            f"{row['1y (%)']:.2f}",
            f"{row['mean (%)']:.2f}",
        )
    except Exception as e:
        return (f"Error: {e}", "", "", "", "")

In [ ]:
iface = gr.Interface(
    fn=predict_ticker,
    inputs=gr.Textbox(label="Ticker e.g. 'TSLA' or 'NVDA'"),
    outputs=[
        gr.Textbox(label="3 Month Change Prediction (%)"),
        gr.Textbox(label="6 Month Change Prediction (%)"),
        gr.Textbox(label="9 Month Change Prediction (%)"),
        gr.Textbox(label="1 Year Change Prediction (%)"),
        gr.Textbox(label="Mean Change Prediction (%)"),
    ],
    title="Stock Price Prediction Model",
    description=" "
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d6f7567b4876962a6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
